In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as transform
from torch.nn.functional import normalize
from PIL import Image
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1. Get training dataset in usable format

- images are in jpg format, need to convert these to PyTorch tensors
- tensors also have to be normalized  

In [2]:
#torchvision.io.read_image()


In [3]:
### class to load and store data
class image_data:   
    def __init__(self):
        self.label_converter = dict()
        self.convert_label_to_int()
        self.train_data    = [] # train data in tensor format
        self.test_data     = [] # test data in tensor format
        self.validate_data = [] # validation data in tensor format
        self.train_labels =  []
        self.test_labels  =  []
        self.image_files = dict()
        self.load_train_data()
        self.load_test_data()
    ### helper function to convert str letters/del/space into numbers 
    def convert_label_to_int(self):
        alphabet = "A/B/C/D/E/F/G/H/I/J/K/L/M/N/O/P/Q/R/S/T/U/V/W/X/Y/Z/del/nothing/space"
        for iii,label in enumerate(alphabet.split("/")):
            self.label_converter[label] = iii
    ### give this an image filename and it will return the transformed tensor 
    def convert_jpg_to_tensor(self,filepath):

        ### this was the method I started using, but it was feasting on RAM, leaving it here in case we want to use any of this later
        """ METHOD 1:
        image = Image.open(filepath) 
        # image to a Torch tensor 
        transform = transforms.Compose([ 
            transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])   """
        """ METHOD 2:
        image = Image.open(filepath) 
        tensor_image = transform.to_tensor(image)
        #print("Tensor size/dim/type = %s/%s/%s"%( tensor_image.size(),tensor_image.dim(), type(tensor_image)))
        
        return tensor_image
        """
        ### there is an integrated method for this that is much faster, but it doesn't normalize
        ### should be returning uint8 
        """ METHOD 3:"""
        return torchvision.io.read_image(filepath)
        

    ### load training data into the instance variable train_data
    def load_train_data(self):
        now = time.time()
        train_path = "datasets/asl_alphabet_train/asl_alphabet_train/"
        train_directories = glob.glob(train_path+"/*")
        n_files = 0
        print(" ----- Loading training dataset -----")

        for dir in train_directories:
            letter = dir.split("/")[-1]
            self.image_files[letter] = []
                
            for image_file in glob.glob(dir+"/*"):
                self.image_files[letter].append(image_file)
                self.train_data.append(self.convert_jpg_to_tensor(image_file))
                self.train_labels.append(self.label_converter[letter])
                n_files +=1
            print("Finished importing %s"%letter)
        print("Done with training dataset - converted %i files. Took %f seconds"%(n_files, np.around(time.time()-now)))
        return
    ### load test data into the instance variable train_data
    def load_test_data(self):
        now = time.time()
        test_path = "datasets/asl_alphabet_test/asl_alphabet_test/"
        test_directories = glob.glob(test_path+"/*")
        n_files = 0
        print("----- Loading test dataset -----")
        for image_file in test_directories:
            letter = image_file.split("_")[-2].split("/")[-1]
            self.image_files[letter] = []
            self.image_files[letter].append(image_file)
            self.test_data.append(self.convert_jpg_to_tensor(image_file))
            self.test_labels.append(self.label_converter[letter])
            n_files +=1
        print("Done with test dataset - converted %i files. Took %f seconds"%(n_files, np.around(time.time()-now,4)))
        return

    

In [ ]:
id = image_data() ## data instance
### the standard method for converting jpg -> tensor uses the commented-out method in convert_jpg_to_tensor.
### With the addition of normalizing tensors option to this method, the kernel crashes
### because this pitiful laptop doesn't have enough RAM. The current method runs, but doesn't normalize.
### Will need to find another way to normalize each tensor


### there is the torch.nn.functional.normalize method, but this didn't like the data type
### that read_image() was outputting. 
### Can change tensor data type with tensor.type(torch.DoubleTensor) 

 ----- Loading training dataset -----
Finished importing R
Finished importing U
Finished importing I
Finished importing N
Finished importing G
Finished importing Z
Finished importing T
Finished importing S
Finished importing A
Finished importing F
Finished importing O
Finished importing H
Finished importing del
Finished importing nothing
Finished importing space
Finished importing M


In [5]:
batch_size = 4 

### now load data into ptytorch
trainloader = torch.utils.data.DataLoader(id.train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(id.test_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

[tensor([[[  0,   0,   1,  ...,   9,   0,   0],
         [  0,   0,  12,  ...,  10,   4,   3],
         [ 10,  16,  40,  ..., 182, 164,  13],
         ...,
         [  7,   7, 120,  ..., 134, 128,  12],
         [  0,   6, 113,  ..., 124, 108,   7],
         [  0,   4,  10,  ...,  14,  11,   4]],

        [[  6,   1,   1,  ...,   3,   0,   2],
         [  0,   0,   8,  ...,   1,   5,   8],
         [  0,   4,  27,  ..., 170, 158,  10],
         ...,
         [  1,   5, 122,  ..., 125, 123,  12],
         [  0,   6, 112,  ..., 119, 109,  12],
         [  1,   6,   9,  ...,  12,  14,  11]],

        [[254, 249, 245,  ..., 251, 247, 248],
         [251, 233, 209,  ..., 190, 211, 222],
         [252, 212, 143,  ..., 236, 255, 163],
         ...,
         [245, 192, 197,  ..., 152, 223, 146],
         [248, 216, 248,  ..., 219, 251, 174],
         [251, 228, 173,  ..., 147, 177, 187]]], dtype=torch.uint8), tensor([[[  0,   0,   0,  ...,  11,   0,   1],
         [  0,   0,  15,  ...,  15,  1